# Cryptopals Challenge Set 4

https://cryptopals.com/sets/4

## 25. Break "random access read/write" AES CTR

https://cryptopals.com/sets/4/challenges/25

In [1]:
import os
from base64 import b64decode
from cryptopals.utils import aes_ecb_decrypt, aes_ctr_decode_encode

with open("input/25.txt") as f:
    cipher_ = b64decode(f.read())
    plaintext25 = aes_ecb_decrypt(cipher_,b"YELLOW SUBMARINE")

KEY25 = os.urandom(16)
NONCE = 0
cipher25 = aes_ctr_decode_encode(plaintext25,KEY25,NONCE)

### Discussion 

Since XOR is commutative, if I gain access to a AES CTR encrypted cipher for which I know the plaintext, I can easily recover the corresponding keystream without knowing the key or the nonce. Here's the generic proof of pricinple:

In [2]:
from cryptopals import bytes_xor

plain_attack = len(cipher25)*b"A"
cipher_attack = aes_ctr_decode_encode(plain_attack,KEY25,NONCE)
keystream_recover = bytes_xor(plain_attack,cipher_attack)
plain_recover = bytes_xor(cipher25,keystream_recover)
print(plain_recover[:200].decode(), "...")

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy ...


### Attack

Assuming I have access to a "random access" read/write API like that described by the challenge and I can read the oringial ciphertext, I would then implement something like the following. The efficiency of the attack depends on the size of the injected chunk, the larger the chunk the fewer call to the re-encrypting API.

In [3]:
from cryptopals.utils import generate_ctr_keystream, bytes_xor

def edit(ciphertext, offset, newtext, key=KEY25, nonce=NONCE):
    keystream = generate_ctr_keystream(key, nonce, offset+len(newtext))   
    newcipher = bytes_xor(newtext,keystream[offset:])
    result = ciphertext[:offset] + newcipher + ciphertext[offset+len(newtext):]
    return result

def break_random_access_read_write_AES_CTR(cipher,chuncksize=1000):
    keystream_recover = b""
    for i in range(len(cipher)//chuncksize+1):
        cipher_edit = edit(cipher, i*chuncksize, chuncksize*b"A")
        keystream_recover += bytes_xor(chuncksize*b"A",cipher_edit[i*chuncksize:(i+1)*chuncksize])
    return bytes_xor(cipher,keystream_recover)

plain25 = break_random_access_read_write_AES_CTR(cipher25)
print(plain25.decode()[:200], "...")

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy ...


## 26. CTR bitflipping

https://cryptopals.com/sets/4/challenges/26

In [34]:
from Cryptodome.Cipher import AES
from cryptopals.utils import aes_ctr_decode_encode
import os

BLOCKSIZE = AES.block_size
KEYSIZE = 32

class profile_functions_26:
    def __init__(self,key=None, nonce=0):
        if not key:
            self.key = os.urandom(KEYSIZE)
        else:
            self.key = key
        self.nonce = nonce
            
    def wrap_userdata(self, data: bytes) -> bytes:
        prefix = b"comment1=cooking%20MCs;userdata="
        suffix = b";comment2=%20like%20a%20pound%20of%20bacon"
        data = data.replace(b";",b"%3B").replace(b"=",b"%3D") # The function should quote out the ";" and "=" characters.
        wrapped = prefix + data + suffix
        return aes_ctr_decode_encode(wrapped,self.key,self.nonce)
    
    def check_for_admin(self, data: bytes, quiet=False) -> bool:
        plaintext = aes_ctr_decode_encode(data,self.key,self.nonce)
        if not quiet:
            print(f"{plaintext=}")
        return b";admin=true;" in plaintext

def make_bitflipping_attack(profile, inject=b";admin=true;") -> bytes:
    a_block = b"A" * BLOCKSIZE
    cipher = profile.wrap_userdata(2*a_block)
    # right justify injection block with padding
    injection = inject.rjust(BLOCKSIZE, b"A")
    flipper = bytes_xor(a_block,injection)
    # flipped block will be 4th block in plain text, it's then left justified to the lenght of the ciphertext
    padded = flipper.rjust(3*BLOCKSIZE, b"\x00").ljust(len(cipher), b"\x00")
    # xor with original encrypter wrapped user data
    cipher_new = bytes_xor(cipher,padded)
    return cipher_new


profile26 = profile_functions_26()

cipher_test = profile26.wrap_userdata(b";admin=true;")
test = profile26.check_for_admin(cipher_test,False)
if not test:
    print("TEST: ';admin=true;' correctly escaped\n")

attack_data = make_bitflipping_attack(profile26,inject=b";admin=true;")
if profile26.check_for_admin(attack_data,False):
    print("ATTACK: injection successfull!")

plaintext=b'comment1=cooking%20MCs;userdata=%3Badmin%3Dtrue%3B;comment2=%20like%20a%20pound%20of%20bacon'
TEST: ';admin=true;' correctly escaped

plaintext=b'comment1=cooking%20MCs;userdata=AAAA;admin=true;AAAAAAAAAAAAAAAA;comment2=%20like%20a%20pound%20of%20bacon'
ATTACK: injection successfull!
